In [2]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [3]:
critics["Lisa Rose"]["Lady in the Water"]


2.5

In [6]:
critics["Toby"]["Snakes on a Plane"]

4.5

In [7]:
#欧几里得距离计算
from math import sqrt
sqrt(pow(4.5-4,2)+pow(1-2,2))

1.118033988749895

In [8]:
1/(1+sqrt(pow(4.5-4,2)+ pow(1-2,2)))

0.4721359549995794

In [9]:
from math import sqrt
def sim_distance(prefs,person1,person2):
    #得到shared_items 的列表
    si = {}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item] = 1
    if len(si) ==0:return 0
    
    # 计算所有差值的平方和
    sum_of_squares = sum([pow(prefs[person1][item] - prefs[person2][item],2)
                         for item in prefs[person1] if item in prefs[person2]])
    
    return 1/(1+sqrt(sum_of_squares))


In [10]:
# example 1 
sim_distance(critics,"Lisa Rose","Gene Seymour")

0.29429805508554946

In [17]:
# 皮尔逊相似度计算函数
# 返回p1 和 p2 的皮尔逊相关系数、
def sim_person(prefs,p1,p2):
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item] = 1
    # 得到列表元素的个数
    n = len(prefs)
    
    #如果两者没有共同之处，则返回1
    if n== 0: return -1
    
    #对所有偏好求和
    sum1 = sum([prefs[p1][item] for item in si])
    sum2 = sum([prefs[p2][item] for item in si])
    
    # 求平方和
    sum1Sq = sum([pow(prefs[p1][item],2) for item in si])
    sum2Sq = sum([pow(prefs[p2][item],2) for item in si])
    
    # 求乘机之和
    pSum = sum([prefs[p1][item]* prefs[p2][item] for item in si])
    
    # 计算皮尔逊的评价值
    num = pSum - (sum1*sum2/n)
    den = sqrt((sum1Sq -pow(sum1,2)/n)*(sum2Sq -pow(sum2,2)/n))
    if den == 0:return 0
    r = num/den
    return r



In [18]:

# example 2
sim_person(critics,"Lisa Rose","Gene Seymour")

0.806983220792075

In [19]:
#  从反映偏好的字典中返回最为匹配者
# 返回的结果的个数和相似度函数均为可选参数
def topMatches(prefs,person,n = 5,similarity = sim_person):
    scores = [(similarity(prefs,person,other),other) 
             for other in prefs if other != person]
    #对列表进行排序，评价最高者排在最前面
    scores.sort()
    scores.reverse()
    return scores[:n]

In [20]:
#  example 3

topMatches(critics,"Toby",n=3)

[(0.987186561133978, 'Michael Phillips'),
 (0.9603016308185575, 'Mick LaSalle'),
 (0.9319189883457425, 'Claudia Puig')]

In [25]:
##  推荐物品
##利用所有人的评价值的加权平均，为某人提供建议
def get_Recomendations(prefs,person,similarity = sim_person):
        totals = {}
        simSums = {}
        for other in prefs:
            if other== person: continue
            sim = similarity(prefs,person,other)
            
            #忽略评价值为零或者评价值小于0的情况
            if sim <=0:continue
            for item in prefs[other]:
                #只对自己还未层看过的影片进行评价
                if item not in prefs[person] or prefs[person][item] ==0:
                    # 相似度*评价值
                    totals.setdefault(item,0)
                    totals[item] += prefs[other][item]*sim
                    
                    #相似度纸盒
                    simSums.setdefault(item,0)
                    simSums[item] +=sim
                    
        # 建立一个归一化列表
        ranking = [(total/simSums[item],item) for item ,total in totals.items()]
        
        # 返回经过排序的列表
        ranking.sort()
        ranking.reverse()
        return ranking
    
    

In [26]:
get_Recomendations(critics,'Toby')

[(3.428458411028444, 'The Night Listener'),
 (2.7933690799738264, 'Lady in the Water'),
 (2.3873335774109146, 'Just My Luck')]

In [27]:
#  匹配商品，想知道哪些商品是相近的
# 定义一个转换函数
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            
            # 将人和 电影调换位置
            
            result[item][person] = prefs[person][item]
    return result


In [28]:
transformPrefs(prefs=critics)

{'Just My Luck': {'Claudia Puig': 3.0,
  'Gene Seymour': 1.5,
  'Lisa Rose': 3.0,
  'Mick LaSalle': 2.0},
 'Lady in the Water': {'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 2.5,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0},
 'Snakes on a Plane': {'Claudia Puig': 3.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 4.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.0,
  'Mick LaSalle': 4.0,
  'Toby': 4.5},
 'Superman Returns': {'Claudia Puig': 4.0,
  'Gene Seymour': 5.0,
  'Jack Matthews': 5.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.5,
  'Mick LaSalle': 3.0,
  'Toby': 4.0},
 'The Night Listener': {'Claudia Puig': 4.5,
  'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 3.0,
  'Michael Phillips': 4.0,
  'Mick LaSalle': 3.0},
 'You, Me and Dupree': {'Claudia Puig': 2.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 3.5,
  'Lisa Rose': 2.5,
  'Mick LaSalle': 2.0,
  'Toby': 1.0}}

In [29]:
# example 4
movies = transformPrefs(prefs=critics)
topMatches(movies,"Superman Returns")

[(0.9168523371635587, 'Lady in the Water'),
 (0.8012590586407953, 'Just My Luck'),
 (0.6579516949597695, 'You, Me and Dupree'),
 (-0.1798471947990544, 'The Night Listener'),
 (-1.145412275435915, 'Snakes on a Plane')]

In [33]:
# example 5 

# 找到和 自己一起*电影的人

get_Recomendations(movies,'Just My Luck')

[(3.672716369447001, 'Jack Matthews'),
 (3.255718276986934, 'Michael Phillips'),
 (3.2301351799398117, 'Toby')]

In [34]:
#  物品过滤
# 构造物品比较的数据集

def calculateSimilarItems(prefs,n=10):
    # 建立字典，以给出与这些电影最为相近的所有其他物品
    result = {}
    # 以电影为中心对些偏好实施倒置处理
    itemPrefs = transformPrefs(prefs)
    
    c = 0
    for item in itemPrefs:
        #针对大数据集更新状态变量
        c +=1
        if c%100 ==0: 
            print(" %d/%d " % (c,len(itemPrefs)))
            
        # 寻找最为相近的物品
        scores = topMatches(itemPrefs,item,n =n,similarity=sim_distance)
        result[item] = scores
    return result



In [35]:
calculateSimilarItems(critics)

{'Just My Luck': [(0.3483314773547883, 'Lady in the Water'),
  (0.32037724101704074, 'You, Me and Dupree'),
  (0.2989350844248255, 'The Night Listener'),
  (0.2553967929896867, 'Snakes on a Plane'),
  (0.20799159651347807, 'Superman Returns')],
 'Lady in the Water': [(0.4494897427831781, 'You, Me and Dupree'),
  (0.38742588672279304, 'The Night Listener'),
  (0.3483314773547883, 'Snakes on a Plane'),
  (0.3483314773547883, 'Just My Luck'),
  (0.2402530733520421, 'Superman Returns')],
 'Snakes on a Plane': [(0.3483314773547883, 'Lady in the Water'),
  (0.32037724101704074, 'The Night Listener'),
  (0.3090169943749474, 'Superman Returns'),
  (0.2553967929896867, 'Just My Luck'),
  (0.1886378647726465, 'You, Me and Dupree')],
 'Superman Returns': [(0.3090169943749474, 'Snakes on a Plane'),
  (0.252650308587072, 'The Night Listener'),
  (0.2402530733520421, 'Lady in the Water'),
  (0.20799159651347807, 'Just My Luck'),
  (0.1918253663634734, 'You, Me and Dupree')],
 'The Night Listener': [

In [38]:
itemsim = calculateSimilarItems(critics)

In [39]:
# 获得推荐

def get_RecommendedItems(prefs,itemMatch,user):
    userRatings = prefs[user]
    scores = {}
    totalSim = {}
    # 循环遍历当前用户评分的物品
    for (item,rating) in userRatings.items():
        
        #循环遍历与当前物品相似的物品
        for ( similarity ,item2) in itemMatch[item]:
            
            # 如果该用户已对当前的电影做过评价，则将其忽略
            if item2 in userRatings:continue
            #评价值与相似度的加权之和
            scores.setdefault(item2,0)
            scores[item2] += similarity*rating
            
            # 全部相似度之和
            totalSim.setdefault(item2,0)
            
            totalSim[item2] += similarity
    #将每个合计值 除以加权和，求出平均值
    ranking = [(score/totalSim[item],item) for item,score in scores.items()]
    
    # 按最高值到最低值排序，返回评分结果
    ranking.sort()
    ranking.reverse()
    return ranking
            

In [40]:
get_RecommendedItems(critics,itemsim,'Toby')

[(3.1667425234070894, 'The Night Listener'),
 (2.9366294028444346, 'Just My Luck'),
 (2.868767392626467, 'Lady in the Water')]

[]

In [42]:

get_RecommendedItems(critics,itemsim,'Jack Matthews')

[(3.5810970647618663, 'Just My Luck')]